# ocr dictionaries for federal sources

In [37]:
import glob

from spacy.lang.de import German
from spacy.lang.fr import French
from spacy.lang.it import Italian
from tika import parser

In [38]:
de_nlp, fr_nlp, it_nlp = German(), French(), Italian()

tika.initVM()

de_path, fr_path, it_path = 'AS-2017-02-21/', 'RO-1948-1998/','RU-1948-1998/'

In [39]:
def get_all_files_in_path(path):
    return [f for f in glob.glob(path + "**/*.xml", recursive=True)]

de_files, fr_files, it_files = get_all_files_in_path(de_path), get_all_files_in_path(fr_path), get_all_files_in_path(it_path)

In [ ]:
def parse_files_and_combine(files):
    text = ''
    for f in files:
        text = ' '.join([text, tika.parser.from_file(f)['content']])
    return text

de_txt, fr_txt, it_txt = parse_files_and_combine(de_files), parse_files_and_combine(fr_files), parse_files_and_combine(it_files)

In [ ]:
# Delete version above, not very efficient
# Backup of text files

with open('de.txt', 'w') as f:
    f.write(de_txt)
    
with open('fr.txt', 'w') as f:
    f.write(fr_txt)
    
with open('it.txt', 'w') as f:
    f.write(it_txt)

In [ ]:
def parse_files(files):
    return [tika.parser.from_file(f)['content'] for f in files]

de_parsed_files, fr_parsed_files, it_parsed_files = parse_files(de_files), parse_files(fr_files), parse_files(it_files)

In [ ]:
de_txt, fr_txt, it_txt = ''.join(de_parsed_files), ''.join(fr_parsed_files), ''.join(it_parsed_files)
# Backup of text files

with open('de.txt', 'w') as f:
    f.write(de_txt)
    
with open('fr.txt', 'w') as f:
    f.write(fr_txt)
    
with open('it.txt', 'w') as f:
    f.write(it_txt)

In [41]:
de_tokens = [token for token in de_nlp(de_txt)]
fr_tokens = [token for token in fr_nlp(fr_txt)]
it_tokens = [token for token in it_nlp(it_txt)]

# https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

In [ ]:
# Remove infrequent words from tokens and convert it to a set()

In [43]:
from spellchecker import SpellChecker

de_spell, fr_spell = SpellChecker(language='de'), SpellChecker(language='fr')
# it_spell =SpellChecker(language='it')
# https://pyspellchecker.readthedocs.io/en/latest/quickstart.html#how-to-build-a-new-dictionary
# https://github.com/hermitdave/FrequencyWords/tree/master/content/2018/it

de_unkown_word = de_spell.unknown(de_tokens)

''